<a href="https://colab.research.google.com/github/AweSamarth/boilerplate-book-recommendation-engine/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [29]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-10-18 17:23:14--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.3’

book-crossings.zip. 100%[===================>]  24.88M  96.3MB/s    in 0.3s    

2023-10-18 17:23:14 (96.3 MB/s) - ‘book-crossings.zip.3’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y
  inflating: BX-Book-Ratings.csv     
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [30]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [31]:
# add your code here - consider creating a new cell for each section of code
df_books=(df_books[(df_books['author']!="") & (df_books['isbn']!="") & (df_books['title']!="")])

df_ratings=df_ratings[df_ratings['user'].isin(df_ratings['user'].value_counts()[df_ratings['user'].value_counts()>199].index)]
df_ratings=df_ratings[df_ratings['isbn'].isin(df_ratings['isbn'].value_counts()[df_ratings['isbn'].value_counts()>99].index)]

# print(df_ratings[df_ratings['isbn']==""])
df_merged= pd.merge(df_books, df_ratings, on='isbn')
print(df_merged)



             isbn                title           author    user  rating
0      0440234743        The Testament     John Grisham  277478     0.0
1      0440234743        The Testament     John Grisham    2977     0.0
2      0440234743        The Testament     John Grisham    3363     0.0
3      0440234743        The Testament     John Grisham    7346     9.0
4      0440234743        The Testament     John Grisham    9856     0.0
...           ...                  ...              ...     ...     ...
13609  0446607657  When the Wind Blows  James Patterson  266056     0.0
13610  0446607657  When the Wind Blows  James Patterson  266226     0.0
13611  0446607657  When the Wind Blows  James Patterson  268330     0.0
13612  0446607657  When the Wind Blows  James Patterson  271284     0.0
13613  0446607657  When the Wind Blows  James Patterson  274308     0.0

[13614 rows x 5 columns]


In [32]:
# matrix=csr_matrix((df_merged['isbn'], df_merged['rating'].values), shape=(2,2))
# print(matrix)
rating_books_pivoted=df_merged.pivot_table(index='title', columns='user', values='rating').fillna(0.0)
print(rating_books_pivoted)

user                                                254     2276    2766    \
title                                                                        
1st to Die: A Novel                                    0.0     0.0     0.0   
A Is for Alibi (Kinsey Millhone Mysteries (Pape...     0.0     0.0     7.0   
A Map of the World                                     0.0     0.0     0.0   
A Painted House                                        0.0     0.0     0.0   
A Prayer for Owen Meany                                0.0     0.0     0.0   
...                                                    ...     ...     ...   
Where the Heart Is (Oprah's Book Club (Paperback))     0.0     0.0     0.0   
While I Was Gone                                       0.0     0.0     0.0   
White Oleander : A Novel (Oprah's Book Club)           0.0     0.0     0.0   
Wicked: The Life and Times of the Wicked Witch ...     0.0     0.0     0.0   
Wild Animus                                            0.0     0

In [35]:
model=NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5, n_jobs=-1)
model.fit(rating_books_pivoted)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

In [37]:
indices=model.kneighbors(rating_books_pivoted.loc[["Where the Heart Is (Oprah's Book Club (Paperback))"]], 5, return_distance=True)
print(indices[1][0])
temparray=[]
print(rating_books_pivoted.index[indices[1]][0])
for index, value in enumerate(rating_books_pivoted.index[indices[1]][0]):
    print((index+1),". ",value)


[94 66 73 64 69]
["Where the Heart Is (Oprah's Book Club (Paperback))"
 'The Lovely Bones: A Novel' "The Pilot's Wife : A Novel"
 'The Joy Luck Club' 'The Notebook']
1 .  Where the Heart Is (Oprah's Book Club (Paperback))
2 .  The Lovely Bones: A Novel
3 .  The Pilot's Wife : A Novel
4 .  The Joy Luck Club
5 .  The Notebook


<ipython-input-37-41782a481e67>:3: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  print(rating_books_pivoted.index[indices[1]][0])
<ipython-input-37-41782a481e67>:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  for index, value in enumerate(rating_books_pivoted.index[indices[1]][0]):


In [ ]:
# print(df_ratings)
# print(df_ratings['isbn'].value_counts())
average_ratings = df_ratings.groupby('isbn')['rating'].mean().reset_index()
# print(average_ratings)
# average_ratings.plot()
# five = NearestNeighbors(n_neighbors=5, algorithm='auto').fit(average_ratings['rating'])



ValueError: ignored

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):


  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

NameError: ignored